In [210]:
import rdflib
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF
import spotlight
import urllib, json
import pandas as pd
from collections import Counter

In [211]:
# load the graph
g = rdflib.Graph()
result = g.parse('kgraph215.ttl', format='turtle')

# setting self-namespaces 
cn = Namespace("http://ai.symptomchecking.org/condtion/")
dbpn = "http://dbpedia.org/resource/"
symptomOf = URIRef(cn.symptomOf)
linksTo = URIRef(cn.linkTo)
signOf = URIRef(cn.signOf)
anatomicOn = URIRef(cn.anatomicOn)
drug = URIRef(cn.drugTo)

# define useful classes
disease = URIRef('http://dbpedia.org/ontology/Disease')
drug = URIRef('http://dbpedia.org/ontology/Drug')
symp = URIRef('http://dbpedia.org/resource/Category:Symptoms_and_signs')
anat = URIRef('http://dbpedia.org/ontology/AnatomicalStructure')

In [212]:
def getting_age_gender(agv,q):
    my_df = pd.DataFrame()
    s=[]
    s0 = [' Baby',' Boy ', ' Boys ',' Girl',' Girls','Children',' Child','Infant','Teenager','Youngster','Youth', 'childhood']
    s1 = [' Over 60 years old','Elderly']
    s2 = ['Middle age']
    s3 = [' Man ',' Men ',' Boy ',' Males ']
    s4 = [' Woman ',' Women ', 'Than males', 'Than men', 'Women are affected', 'Females are affected', 'Most often in females', 'and females', 'infected female', 'infected females', 'females', 'female', 'girl', 'girls', 'pregnancy', 'Pregnancy']
    if q == 0:
        my_df = pd.DataFrame(columns=['Disease','AgeGroup'])
        if agv==0:
            agv = 'young'
            s=s0
        elif agv==1:
            agv = 'old'
            s=s1
        else:
            s=s2
            agv = 'Middle age'
    if q == 1:
        my_df = pd.DataFrame(columns=['Disease','Gender'])
        if agv==0:
            agv = 'Male'
            s=s3
        else:
            agv = 'Female'
            s=s4
    j=0
    querystring = "SELECT DISTINCT ?d WHERE {"
    for xqi in s:
        if (j<len(s)-1):
            querystring = querystring + "{\
            ?d <http://purl.org/dc/elements/1.1/abstract> ?a . FILTER regex(str(?a), '"+xqi+"')} UNION {\
            ?d <http://purl.org/dc/elements/1.1/abstract> ?a . FILTER regex(str(?a), '"+xqi.lower()+"')} UNION"
        else:
            querystring = querystring + "{\
            ?d <http://purl.org/dc/elements/1.1/abstract> ?a . FILTER regex(str(?a), '"+xqi+"')} UNION {\
            ?d <http://purl.org/dc/elements/1.1/abstract> ?a . FILTER regex(str(?a), '"+xqi.lower()+"')}"
        j=j+1
    querystring = querystring + "}"
    #print (querystring)
    qres = g.query(querystring)
    i=0
    for row in qres:
        d = str(row.asdict()['d'].toPython())
        my_df.loc[i] = [d,agv]
        i=i+1
    return my_df

In [213]:
def getting_position():
    my_df = pd.DataFrame(columns=['Disease', 'Position'])
    querystring = "SELECT DISTINCT ?d ?a WHERE {"
    querystring = querystring + "?d <"+anatomicOn+"> ?a .}"
    #print querystring
    qres = g.query(querystring)
    i=0
    for row in qres:
        d = str(row.asdict()['d'].toPython())
        a = str(row.asdict()['a'].toPython())
        my_df.loc[i] = [d,a]
        i=i+1
    my_df = my_df.groupby('Disease').agg({'Position': ', '.join}).reset_index()
    return my_df

In [214]:
def getting_sym_dis():
    my_df = pd.DataFrame(columns=['Disease', 'Symptom'])
    querystring = "SELECT DISTINCT ?d ?s WHERE {"
    querystring =  querystring+"{?s <"+signOf+"> ?d} UNION {?s <"+symptomOf+"> ?d} .}"
    #print querystring
    qres = g.query(querystring)
    i=0
    for row in qres:
        d = str(row.asdict()['d'].toPython())
        s = str(row.asdict()['s'].toPython())
        my_df.loc[i] = [d,s]
        i=i+1
    my_df = my_df.groupby('Disease').agg({'Symptom': ', '.join}).reset_index()

            # me_df = pd.merge(me_df, my_df, on=['Disease'])
    return my_df

# generation age parameters

In [215]:
age0_gender_0_df = getting_age_gender(0,0)

In [216]:
age1_gender_0_df = getting_age_gender(1,0)

In [217]:
age2_gender_0_df = getting_age_gender(2,0)

In [218]:
age_merge = pd.concat([age0_gender_0_df, age1_gender_0_df, age2_gender_0_df])

In [219]:
age_merge = age_merge.groupby('Disease').agg({'AgeGroup': ', '.join}).reset_index()

In [220]:
age_merge.to_csv("ageTestP.csv", encoding='utf-8')

# Age generation end

# Position and symptom parameter generation 

In [221]:
redf = pd.merge(getting_sym_dis(), getting_position(), on='Disease')

In [222]:
redf.to_csv("symPosTestP.csv", encoding='utf-8')

# Position and symptom generation end 

In [223]:
male_df = getting_age_gender(0,1)

In [224]:
female_df = getting_age_gender(1,1)

In [225]:
gender_merge = pd.concat([male_df, female_df])

In [226]:
gender_merge = gender_merge.groupby('Disease').agg({'Gender': ', '.join}).reset_index()

In [227]:
gender_merge.to_csv("genderTestP.csv", encoding='utf-8')

In [228]:
redf.iloc[0]['Disease']

'http://ai.symptomchecking.org/condtion/Abdominal_aortic_aneurysm'

In [229]:
print (gender_merge[gender_merge['Disease']==redf.iloc[0]['Disease']])

                                             Disease        Gender
0  http://ai.symptomchecking.org/condtion/Abdomin...  Male, Female


In [162]:
p_df = pd.DataFrame(columns=['Disease', 'Symptom','Position', 'Age', 'Gen'])
for i in range (0,len(redf)):
    ck = redf.iloc[i]['Disease']
    cp = redf.iloc[i]['Position']
    sp = redf.iloc[i]['Symptom']
    ge = 'any'
    ag = 'any'
    for j in range (0, len(gender_merge)):
        if gender_merge.iloc[j]['Disease']==ck:
            ge = gender_merge.iloc[j]['Gender']
            if len(ge.split(','))>1:
                ge = 'any'
    for k in range (0, len(age_merge)):
        if age_merge.iloc[k]['Disease']==ck:
            ag = age_merge.iloc[k]['AgeGroup']
    p_df.loc[i] = [ck, sp, cp, ag, ge]

In [164]:
len(p_df)

184

In [230]:
p_df.to_csv("testSeeds.csv", encoding='utf-8')

In [231]:
test_df = pd.DataFrame(columns=['Disease', 'Symptom','Position', 'Age', 'Gen'])

In [232]:
import random

In [233]:
ndp = ['Fever','Cough','Headache','Infection','Dizzy','Breath','Sick', 'Temperature','Tiredness','Vomiting']
for i in range (0,len(p_df)):
    ck = p_df.iloc[i]['Disease']
    cp = p_df.iloc[i]['Position']
    if len(cp.split(','))>3:
        rnb = random.randint(1,len(cp.split(','))-2)
        cp = cp.split(',')[0:rnb]
    sp = p_df.iloc[i]['Symptom']
    rnd = random.randint(0,5)
    sp = sp + ndp[rnd] 
    rnd = random.randint(5,9)
    sp = sp + ndp[rnd]  
    if len(sp.split(','))>5:
        rnb = random.randint(2,len(sp.split(','))-3)
        sp = sp.split(',')[0:rnb]
    ap = p_df.iloc[i]['Age']
    if ap == 'any':
        foo = ['Yong','Elder','Middel age']
        ap = random.choice(foo)
    gp = p_df.iloc[i]['Gen']
    if gp == 'any':
        foo = ['male','female']
        gp = random.choice(foo)
    test_df.loc[i] = [ck, sp, cp, ap, gp]   

In [234]:
p_df.iloc[0]['Gen']

'any'

In [235]:
test_df.to_csv("testdata1.csv", encoding='utf-8')